In [ ]:
import cv2
import numpy as np
import os

In [75]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [76]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [77]:
def nonmax_suppression(image):

    NONMAX_SUPPRESSION_KERNEL = 16
    STRIDE = 8
    
    result = np.zeros((image.shape))
    
    _image = np.lib.stride_tricks.sliding_window_view(image, NONMAX_SUPPRESSION_KERNEL, 1)[:,::STRIDE]
    _image = np.lib.stride_tricks.sliding_window_view(_image, NONMAX_SUPPRESSION_KERNEL, 0)[::STRIDE,:]

    _image = np.reshape(_image,(*_image.shape[:2],-1))
    _image = np.argmax(_image, axis=2)

    mg = np.mgrid[0:_image.shape[0],0:_image.shape[1]]

    # y offset: mg[0], x offset: mg[1]
    index_y = _image // NONMAX_SUPPRESSION_KERNEL + mg[0] * STRIDE 
    index_x = _image % NONMAX_SUPPRESSION_KERNEL + mg[1] * STRIDE 
    index_x, index_y = index_x.flatten(), index_y.flatten()


    for i, j in zip(index_y, index_x):
        result[i][j]=image[i][j]  
     
    return  result

In [94]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 5
    THRESHOLD = 500
    K = 0.04

    #  Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (3,3),3)
    # gradient
    I_x = np.gradient(Gau_img, axis=1) 
    I_y = np.gradient(Gau_img, axis=0) 


    # Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    
    # Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255

    # Nonmax Suppression
    R = nonmax_suppression(R)
    
    # 濾邊界
    R[:20 , :] = 0
    R[: ,:20 ] = 0
    R[-20:, :] = 0
    R[:,-20: ] = 0
    # return mask
    return R
        

In [217]:
def feature_descriptor(feature_mask):
    DESCRIPTOR_SIZE = 40
    KERNEL = 5

    # descriptor size: 40*40
    feature_point_x, feature_point_y = np.nonzero(feature_mask)
    index_to_get = [KERNEL//2 + i * KERNEL for i in range(DESCRIPTOR_SIZE//KERNEL)]
    # 5*5 sum/mean -> 8*8(descriptor matrix)

    descriptors = []
    positions = []
    for x, y in zip(feature_point_x, feature_point_y):
        

        # 取-20~19之間的值，做boxfilter
        big_matrix = cv2.boxFilter(
            feature_mask[x-DESCRIPTOR_SIZE//2:x+DESCRIPTOR_SIZE//2, y-DESCRIPTOR_SIZE//2:y+DESCRIPTOR_SIZE//2],
             -1, (KERNEL, KERNEL),
             )

        descriptor = np.zeros((8, 8))

        for i in range(DESCRIPTOR_SIZE//KERNEL):
            for j in range(DESCRIPTOR_SIZE//KERNEL):
                descriptor[i][j] = big_matrix[index_to_get[i]][index_to_get[j]]

        descriptors.append(descriptor)
        positions.append([x, y])

    return descriptors, positions

In [230]:
def matching(descriptors_1, positions_1, descriptors_2, positions_2):
    LOW_THERESHOLD = 30

    match_point = []

    for ds1, ps1 in zip(descriptors_1, positions_1):
        min_dis = 99999
        min_pos = [-1, -1]

        for ds2, ps2 in zip(descriptors_2, positions_2):
            dis = np.linalg.norm(ds1-ds2)

            if(dis<min_dis):
                min_dis = dis
                min_pos = [*ps1, *ps2]
                
        if(min_dis<LOW_THERESHOLD):
            match_point.append(min_pos)
    return match_point 
    # [pos1_x, pos1_y, pos2_x, pos2_y]
    

In [231]:
a = load_images_from_folder('img/test')
R = detect_Harris_corner(a[0])
R2 = detect_Harris_corner(a[1])
ds1, ps1 = feature_descriptor(R)
ds2, ps2 = feature_descriptor(R2)
mp = matching(ds1, ps1, ds2, ps2)

img_concat = np.concatenate((a[0], a[1]), axis=1)
cv2.line()